# Import

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.metrics import cohen_kappa_score, classification_report, make_scorer
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, OneHotEncoder, Normalizer, MinMaxScaler, RobustScaler, MaxAbsScaler
from sklearn.model_selection import train_test_split, cross_validate, StratifiedShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from catboost import CatBoostClassifier
import seaborn as sns

pd.set_option('display.max_rows', 500)
random_state = 42
np.random.seed(random_state)
data_filepath = Path('./data')
# data_filepath = Path('/kaggle/input/child-mind-institute-problematic-internet-use')
KAPPA_SCORER = make_scorer(
    cohen_kappa_score, 
    greater_is_better=True, 
    weights='quadratic',
)

# Data

In [2]:
!du -hs $data_filepath/*
train_df = pd.read_csv(data_filepath / 'train.csv')
test_df = pd.read_csv(data_filepath / 'test.csv')
train_df.shape, test_df.shape

6.2G	data/child-mind-institute-problematic-internet-use.zip
 12K	data/data_dictionary.csv
4.0K	data/sample_submission.csv
7.9M	data/series_test.parquet
6.3G	data/series_train.parquet
8.0K	data/test.csv
924K	data/train.csv


((3960, 82), (20, 59))

In [3]:
%%time
train_ts = pd.read_parquet('data/series_train.parquet')
train_ts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 314569149 entries, 0 to 314569148
Data columns (total 14 columns):
 #   Column               Dtype   
---  ------               -----   
 0   step                 uint32  
 1   X                    float32 
 2   Y                    float32 
 3   Z                    float32 
 4   enmo                 float32 
 5   anglez               float32 
 6   non-wear_flag        float32 
 7   light                float32 
 8   battery_voltage      float32 
 9   time_of_day          int64   
 10  weekday              int8    
 11  quarter              int8    
 12  relative_date_PCIAT  float32 
 13  id                   category
dtypes: category(1), float32(9), int64(1), int8(2), uint32(1)
memory usage: 15.2 GB
CPU times: user 20.3 s, sys: 18.4 s, total: 38.7 s
Wall time: 35.4 s
